# Initalization

In [ ]:
from bs4 import BeautifulSoup
import re
import json
from IPython.display import display, Image

In [ ]:
#Load in file created with  SINGLEFILE  extension
with open('/content/(20_) Hollis Queens NY _ Facebook (11_13_2024 2_47_24 PM).html', 'r', encoding='utf-8') as file:
    html_content = file.read()

soup = BeautifulSoup(html_content, 'html.parser')

In [ ]:
#Parse through soup and separate all post data
captionsWithSep=[]
outer_divs = soup.find_all('div', class_='x1yztbdb x1n2onr6 xh8yej3 x1ja2u2z')
for outer_div in outer_divs:
    if outer_div:
      captionsWithSep.append(outer_div.get_text(strip=True,separator="$$"))

# Data extraction

Extract Poster Names

In [ ]:
def extactPosterNames(data_list):
    """
    Extracts the name of poster held within the 33rd separation '$$' as the separator
    from each list member

    Args:
        data_list: A list of strings, where each string contains each post's data separated by '$$'.

    Returns:
        A list where each member is the name of the poster.
    """

    extracted_words = []
    for data_str in data_list:
        parts = data_str.split("$$")
        if len(parts) >= 33:  # Check if the 33rd separation exists
            extracted_word = parts[33]  # Name is always held in 33rd position
            extracted_words.append(extracted_word)
        else:
            extracted_words.append(None)

    return extracted_words

extracted_words = extactPosterNames(captionsWithSep)
#Manual edit for post not pertaining to current project
del extracted_words[25]
print(extracted_words)
print(len(extracted_words))

Extract post text

In [ ]:
def extract_post(entries):
  """
  Extracts data from a list of entries, where each entry is a string
  containing data separated by "$$".

  For each entry, it finds the data within a separator which says "Shared with Private group" and saves the data at the following separator.
  If there is aditional data held after that separtor, it saves that data as well.

  Args:
    entries: A list of strings, where each string is all that post's data.

  Returns:
    A list of strings, where each string contains the extracted data.
  """

  extracted_data = []
  for entry in entries:
    parts = entry.split("$$")
    for i, part in enumerate(parts):
      if "Shared with Private group" in part:
        try:

          if len(parts[i+1])<3:
            extracted_data.append("NULL")
          elif len(parts[i+3])>3:
            dataToAppend=parts[i+1]+" "+parts[i+2]+" "+parts[i+3]
            extracted_data.append(dataToAppend)
          elif len(parts[i+2])>3:
            dataToAppend=parts[i+1]+" "+parts[i+2]
            extracted_data.append(dataToAppend)
          else:
            dataToAppend=parts[i+1]
            extracted_data.append(dataToAppend)

          break  #Move to the next entry
        except IndexError:
          # if its at the end, append blank
          extracted_data.append("")
          break
    else:
      # If "Shared with Private group" is not found in the entry
      extracted_data.append("")

  return extracted_data


extracted_results = extract_post(captionsWithSep)


print(extracted_results)
#Manual edit for post not pertaining to current project
del extracted_results[25]
print(len(extracted_results))

Extract image data

In [ ]:
bse64Img=[]
img_tags = soup.find_all('img', class_=re.compile('x1ey2m1c xds687c x5yr21d x10l6tqk x17qophe x13vifvy xh8yej3'))

i = 0  # Initialize the index variable
while i < len(img_tags):
    img_tag = img_tags[i]
    img_url = img_tag.get('src')
    alt_text = img_tag.get('alt', '')

    if alt_text:
        bse64Img.append(img_url)
        i += 1  # Move to the next image
    else:
        # Check if the next image exists to avoid IndexError
        if i + 1 < len(img_tags):
            next_img_url = img_tags[i + 1].get('src')
            bse64Img.append([img_url, next_img_url])
            i += 2  # Skip the next image (move ahead by 2)
        else:
            # If no next image exists, just move to the next image
            i += 1




# Combining all data into Json

In [ ]:
bse64Img=[]
img_tags = soup.find_all('img', class_=re.compile('x1ey2m1c xds687c x5yr21d x10l6tqk x17qophe x13vifvy xh8yej3'))

i = 0  # Initialize the index variable
while i < len(img_tags):
    img_tag = img_tags[i]
    img_url = img_tag.get('src')
    alt_text = img_tag.get('alt', '')

    if alt_text:
        bse64Img.append(img_url)
        i += 1
    else:
        # Check if the next image exists to avoid IndexError
        if i + 1 < len(img_tags):
            next_img_url = img_tags[i + 1].get('src')
            bse64Img.append([img_url, next_img_url])
            i += 2  # Skip the next image (move ahead by 2)
        else:
            # No image, move on
            i += 1




In [ ]:
# Display the JSON file
with open("output.json", "r") as json_file:
    allDataDict = json.load(json_file)

for entry_key, entry_data in allDataDict.items():
    print(f"Entry {entry_key}:")
    print(f"  Group Name: {entry_data['groupName']}")
    print(f"  Poster: {entry_data['poster']}")
    print(f"  Text: {entry_data['text']}")

    # Decode the base64 image data
    picData = entry_data['picData']
    if isinstance(picData, list):
        for idx, pic in enumerate(picData):
          display(Image(url=pic))
    else:
      display(Image(url=picData))
